In [ ]:
# Mohsen Dorraki
# 18/04/2023
# Citrulline vs Alpha diversity
######################################### requirement #########################################

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import seaborn as sb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import signal
import tensorflow as tf
from scipy.stats import pearsonr
###############################################################################################


In [ ]:
######################################### Reading data #########################################

#df2 = pd.read_csv() 

In [ ]:
######################################### Plot data #########################################
fig = plt.figure(figsize = (8, 6))
sns.regplot(data = df2, x = 'Citrulline', y ='AlphaDiversity')
plt.title('Citrulline vs Alpha Diversity ', weight='bold', fontsize = 15)
plt.ylabel('Alpha Diversity', weight='bold', fontsize = 12)
plt.xlabel('Citrulline ', weight='bold', fontsize = 12)
plt.show()

In [ ]:
######################################### Plot individuals #########################################
#We drop patients with low number of data
LowSampleIDs = [3, 6, 8, 9, 11, 16, 33, 34]
df3 = df2[~df2['ID'].isin(LowSampleIDs)]
df3 = df3.dropna(subset=['Citrulline', 'AlphaDiversity'])

df3['NewID'] = 1

# Assign unique number to each group
for i in range(1, len(df3)):
    if df3.iloc[i]['ID'] == df3.iloc[i-1]['ID']:
        df3.at[df3.index[i], 'NewID'] = df3.iloc[i-1]['NewID']
    else:
        df3.at[df3.index[i], 'NewID'] = df3.iloc[i-1]['NewID'] + 1


# create 35 subplots in a 7x5 grid
fig, axs1 = plt.subplots(7, 4, figsize=(20, 30))
# loop over each subplot
for i, ax1 in enumerate(axs1.flatten(), 1):
    df = df3[df3['NewID'] == i]
    df = df.sort_values('TreatmentDay')  # sort based on treatment days

    x_data = df['TreatmentDay']
    y_data = df['Citrulline']
    z_data = df['AlphaDiversity']

    # plot the x-y and x-z relationships in the subplot
    ax1.set_xlim([-3, 95])  # limit x axis
    ax1.set_ylim([0, 60])  # limit y axis
    ax1.scatter(x_data, y_data, color='purple', label='Citrulline')
    ax1.plot(x_data, y_data, color='purple')
    ax1.set_xlabel('Treatment day', fontsize=20)
    ax1.set_ylabel('Citrulline', fontsize=20, color='purple')

    ax2 = ax1.twinx()
    ax2.set_ylim([0, 6])  # limit y axis
    ax2.scatter(x_data, z_data, color='green', label='Alpha Diversity x 10')
    ax2.plot(x_data, z_data, color='green')
    ax2.set_ylabel('Alpha diversity', fontsize=20, color='green')

    ax1.set_title(f"Patient {i}", fontsize=24)  # set the title of the subplot
    ax1.tick_params(axis='both', labelsize=16)  # increase tick label font size
    ax1.tick_params(axis='both', which='major', labelsize=16)  # increase major tick label font size
    ax2.tick_params(axis='both', labelsize=16)  # increase tick label font size

    
    # Set y-axis color to match the lines
    ax1.spines['left'].set_color('purple')
    ax2.spines['right'].set_color('green')
    ax1.yaxis.label.set_color('purple')
    ax2.yaxis.label.set_color('green')
    ax1.tick_params(axis='y', colors='purple')
    ax2.tick_params(axis='y', colors='green')

    plt.tight_layout()

# display and save the plot
plt.savefig('Filteredplot.png', dpi=300, bbox_inches='tight')
plt.show()




In [ ]:

######################################### Cross-correlation #########################################

def ccf_values(series1, series2):
    p = series1
    q = series2
    p = (p - np.mean(p)) / (np.std(p) * len(p))
    q = (q - np.mean(q)) / (np.std(q))  
    c = np.correlate(p, q, 'full')
    return c

def ccf_plot(lags, ccf, i):
    #fig, ax =plt.subplots(figsize=(9, 6))
    ax.plot(lags, ccf)
    ax.axhline(-2/np.sqrt(23), color='red', label='5 percent confidence interval')
    ax.axhline(2/np.sqrt(23), color='red')
    ax.axvline(x = 0, color = 'black', lw = 1)
    ax.axhline(y = 0, color = 'black', lw = 1)
    ax.axhline(y = np.max(ccf), color = 'blue', lw = 1, 
    linestyle='--', label = 'highest +/- correlation')
    ax.axhline(y = np.min(ccf), color = 'blue', lw = 1, 
    linestyle='--')
    ax.set(ylim = [-1, 1])
    ax.set_title(f"Patient {i}", fontsize=24)  # set the title of the subplot
    ax.set_ylabel('Correlation Coefficients', weight='bold', fontsize = 20)
    ax.set_xlabel('Time Lags', weight='bold', fontsize = 20)
    ax.tick_params(axis='both', labelsize=16)  # increase tick label font size
    ax.tick_params(axis='both', which='major', labelsize=16)  # increase major tick label font size
    plt.legend()
    plt.tight_layout()   



# create 28 subplots in a 7x4 grid
fig, axs = plt.subplots(7, 4, figsize=(20, 30))
correlation_array = []
p_value_array = []
# loop over each subplot
for i, ax in enumerate(axs.flatten(),1):
    df=df3[df3['NewID'] == i]
    df= df.sort_values('TreatmentDay')  # sort based treatment days

    # generate data for each subplot
    x_data = df['TreatmentDay']
    y_data = df['Citrulline']
    z_data = df['AlphaDiversity']*10


    ccf = ccf_values(y_data, z_data)
    lags = signal.correlation_lags(len(y_data), len(z_data))
    ccf_plot(lags, ccf, i)

    correlation, p_value = pearsonr(y_data, z_data)
    correlation_array.append(correlation)
    p_value_array.append(p_value)


# display the plot
plt.savefig('CrossCorr.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
######################################### Correlation coefficients #########################################

    print("Correlation coefficient:", correlation_array)
    print("P-value:", p_value_array)

In [ ]:
######################################### Machine learning #########################################


# Regression

from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score


X = df3[['Citrulline']] # input variable
y = df3[['AlphaDiversity']] # output variable


# split data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create a Linear Regression model and fit it to the training data
model = LinearRegression()
model.fit(X_train, y_train)


# evaluate the model on the testing set
y_pred = model.predict(X_test)
# evaluate the model on the testing set
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the results
print('RMSE:', rmse)
print(f'Mean absolute error: {mae:.2f}')
print('MSE:', mse)
print(f'R-squared value: {r2:.2f}')
# use the model to predict the value of "a" for a new value of "b"
new_b = [[20]] # a new value of "b" to predict the value of "a"
new_a = model.predict(new_b)
print('Predicted value of "a" for b=', new_b, ':', new_a)


